In [1]:
import brainbow_model_mouseline as model

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
variables = {
    "n_channels": 5,
    "p_cre": 0.8,
    "seed": 46,
    "num_configurations": 500,
    "tag_weight": 0.7,
    "base_marker_weight": 1.0,
    "min_base_markers": 0,
    "max_tags_per_marker": 2,
}
# this is a great result -- seed 40 !

print("Finding optimal cassette configuration...")
print("This may take a few minutes...")

# Use custom configuration
custom_config = model.BrainbowConfig(
    base_markers=["mScarlet3", "mBaojin", "hfYFP", "mNeonGreen"],
    tags=["HSV", "TD", "ALFA", "NWS"],
    max_tags_per_marker=2,
)

# Find optimal configuration for 3 channels
optimal_config = model.find_optimal_cassette_configuration(
    num_configurations=variables["num_configurations"],
    p_cre=variables["p_cre"],
    target_channels=variables["n_channels"],
    seed=variables["seed"],
    verbose=False,
    base_marker_weight=variables["base_marker_weight"],
    tag_weight=variables["tag_weight"],
    min_base_markers=variables["min_base_markers"],
    config=custom_config,  # Ensure at least 2 base markers are used
)

# Analyze cell labeling with the optimal panel
best_panel = optimal_config["best_panel_info"]["best_panels"][0]
labeling_analysis = model.analyze_cell_labeling(optimal_config["best_df"], best_panel)

model.print_optimal_configuration_report(
    labeling_analysis, variables, best_panel, optimal_config
)

Finding optimal cassette configuration...
This may take a few minutes...
=== CELL LABELING ANALYSIS ===
Panel: ['ALFA', 'hfYFP', 'mBaojin', 'mNeonGreen', 'mScarlet3']
Percent of Cre-labeled cells: 80.0%
Total cells labeled in panel: 100.0%
Cells not labeled in panel: 0.0%
Number of unique combinations: 15

=== MARKER COMBINATION DISTRIBUTION ===
mNeonGreen + mScarlet3: 11.1%
ALFA + hfYFP: 7.1%
ALFA + hfYFP + mBaojin: 7.1%
ALFA + hfYFP + mNeonGreen: 7.1%
ALFA + mBaojin: 7.1%
ALFA + mBaojin + mScarlet3: 7.1%
hfYFP + mBaojin: 7.1%
hfYFP + mScarlet3: 7.1%
mBaojin + mNeonGreen: 7.1%
ALFA + hfYFP + mScarlet3: 5.3%
ALFA + mBaojin + mNeonGreen: 5.3%
hfYFP + mNeonGreen: 5.3%
mBaojin + mScarlet3: 5.3%
mNeonGreen: 5.3%
mScarlet3: 5.3%

=== OPTIMAL CONFIGURATION DETAILS ===
Best entropy: 3.499 bits
Entropy per channel: 0.700 bits

Optimal Cassette A:
  Null: mScarlet3-TD
  Bit1: mNeonGreen-TD
  Bit2: hfYFP-HSV
  Bit3: mBaojin-ALFA

Optimal Cassette B:
  Null: mNeonGreen
  Bit1: mBaojin-NWS
  Bit2:

In [13]:
full_analysis = model.analyze_cell_labeling(optimal_config["best_df"], set(optimal_config['config_used'].base_markers + optimal_config['config_used'].tags))
model.print_optimal_configuration_report(
    full_analysis, variables, best_panel, optimal_config
)

=== CELL LABELING ANALYSIS ===
Panel: ['ALFA', 'HSV', 'NWS', 'TD', 'hfYFP', 'mBaojin', 'mNeonGreen', 'mScarlet3']
Percent of Cre-labeled cells: 80.0%
Total cells labeled in panel: 100.0%
Cells not labeled in panel: 0.0%
Number of unique combinations: 15

=== MARKER COMBINATION DISTRIBUTION ===
TD + mNeonGreen + mScarlet3: 11.1%
ALFA + HSV + hfYFP: 7.1%
ALFA + NWS + mBaojin: 7.1%
ALFA + TD + hfYFP + mNeonGreen: 7.1%
ALFA + hfYFP + mBaojin: 7.1%
ALFA + mBaojin + mScarlet3: 7.1%
HSV + NWS + hfYFP + mBaojin: 7.1%
HSV + hfYFP + mScarlet3: 7.1%
NWS + TD + mBaojin + mNeonGreen: 7.1%
ALFA + TD + hfYFP + mScarlet3: 5.3%
ALFA + mBaojin + mNeonGreen: 5.3%
HSV + hfYFP + mNeonGreen: 5.3%
NWS + TD + mBaojin + mScarlet3: 5.3%
TD + mNeonGreen: 5.3%
TD + mScarlet3: 5.3%

=== OPTIMAL CONFIGURATION DETAILS ===
Best entropy: 3.499 bits
Entropy per channel: 0.700 bits

Optimal Cassette A:
  Null: mScarlet3-TD
  Bit1: mNeonGreen-TD
  Bit2: hfYFP-HSV
  Bit3: mBaojin-ALFA

Optimal Cassette B:
  Null: mNeonGre

In [9]:
panel = set(optimal_config['config_used'].base_markers)
df = model.outcome_table(
        0.8,
        optimal_config["best_cassette_a"],
        optimal_config["best_cassette_b"],
        1,
        1,
        1,
    )
base_labeling_analysis = model.analyze_cell_labeling(df, panel)
print(
    f"Total cells labeled with base markers: {base_labeling_analysis['total_labeled_percent']:.1f}%"
)
print(
    f"Cells not labeled with base markers: {base_labeling_analysis['unlabeled_percent']:.1f}%"
)
print(
    f"Number of unique base marker combinations: {base_labeling_analysis['num_combinations']}"
)
for _, row in base_labeling_analysis["combination_distribution"].iterrows():
    if len(row["Marker_Combination"]) == 0:
        combo_name = "No markers (unlabeled)"
    else:
        combo_name = " + ".join(sorted(row["Marker_Combination"]))
    print(f"{combo_name}: {row['Percentage']:.1f}%")


Total cells labeled with base markers: 100.0%
Cells not labeled with base markers: 0.0%
Number of unique base marker combinations: 10
hfYFP + mBaojin: 14.2%
hfYFP + mNeonGreen: 12.4%
hfYFP + mScarlet3: 12.4%
mBaojin + mNeonGreen: 12.4%
mBaojin + mScarlet3: 12.4%
mNeonGreen + mScarlet3: 11.1%
hfYFP: 7.1%
mBaojin: 7.1%
mNeonGreen: 5.3%
mScarlet3: 5.3%


In [16]:
panel = set(optimal_config['config_used'].base_markers)
df = model.outcome_table(
        0,
        optimal_config["best_cassette_a"],
        optimal_config["best_cassette_b"],
        1,
        1,
        1,
    )
base_labeling_analysis = model.analyze_cell_labeling(df, panel)
print(
    f"Total cells labeled with base markers: {base_labeling_analysis['total_labeled_percent']:.1f}%"
)
print(
    f"Cells not labeled with base markers: {base_labeling_analysis['unlabeled_percent']:.1f}%"
)
print(
    f"Number of unique base marker combinations: {base_labeling_analysis['num_combinations']}"
)
for _, row in base_labeling_analysis["combination_distribution"].iterrows():
    if len(row["Marker_Combination"]) == 0:
        combo_name = "No markers (unlabeled)"
    else:
        combo_name = " + ".join(sorted(row["Marker_Combination"]))
    print(f"{combo_name}: {row['Percentage']:.1f}%")


Total cells labeled with base markers: 100.0%
Cells not labeled with base markers: 0.0%
Number of unique base marker combinations: 10
mNeonGreen + mScarlet3: 100.0%
hfYFP: 0.0%
hfYFP + mBaojin: 0.0%
hfYFP + mNeonGreen: 0.0%
hfYFP + mScarlet3: 0.0%
mBaojin: 0.0%
mBaojin + mNeonGreen: 0.0%
mBaojin + mScarlet3: 0.0%
mNeonGreen: 0.0%
mScarlet3: 0.0%


In [17]:
panel = set(optimal_config['config_used'].base_markers)
df = model.outcome_table(
        1,
        optimal_config["best_cassette_a"],
        optimal_config["best_cassette_b"],
        1,
        1,
        1,
    )
base_labeling_analysis = model.analyze_cell_labeling(df, panel)
print(
    f"Total cells labeled with base markers: {base_labeling_analysis['total_labeled_percent']:.1f}%"
)
print(
    f"Cells not labeled with base markers: {base_labeling_analysis['unlabeled_percent']:.1f}%"
)
print(
    f"Number of unique base marker combinations: {base_labeling_analysis['num_combinations']}"
)
for _, row in base_labeling_analysis["combination_distribution"].iterrows():
    if len(row["Marker_Combination"]) == 0:
        combo_name = "No markers (unlabeled)"
    else:
        combo_name = " + ".join(sorted(row["Marker_Combination"]))
    print(f"{combo_name}: {row['Percentage']:.1f}%")


Total cells labeled with base markers: 100.0%
Cells not labeled with base markers: 0.0%
Number of unique base marker combinations: 10
hfYFP + mBaojin: 22.2%
hfYFP: 11.1%
hfYFP + mNeonGreen: 11.1%
hfYFP + mScarlet3: 11.1%
mBaojin: 11.1%
mBaojin + mNeonGreen: 11.1%
mBaojin + mScarlet3: 11.1%
mNeonGreen + mScarlet3: 11.1%
mNeonGreen: 0.0%
mScarlet3: 0.0%
